Have a Taste of DNN on the Shoulder of Theano
====

![image](fruit.png)

## Outline
* Introduction to Theano
* Softmax regression
* Highlights of DNN
* Neural network
    * Multiple layer perception
    * Forward propagation
    * Backward propagation
* Sparse autoencoder
    * Autoencoder
    * Sparse autoencoder
* Building deep networks for classification
    * Model structure
    * Pre-training
    * Fine-tuning
    * Experimental results

## Introduction to Theano
Theano is Python library that allows you to define, evaluate and optimize math expressions.
* Efficient symbolic differentiation
* Efficient handling of matrices
* Tight integration of NumPy
* Dynamic C code generate
* Transparent use of GPU

#### Fast to develop and fast to run
![image](fast.png)

#### Machine learning libraries built on top of Theano:
* Pylearn2
    * great flexibility and a good choice for trying out ML ideas
* PyMC3
    * Probabilistic programming; building statistical Bayesian models
* Sklearn-theano
    * Easy-to-use deep learning tool
* Lasagne
    * Lightweight library to build neural networks

#### Models that have been built with Theano:
* Neural networks
* Convolutional Neural Networks (CNN)
* Recurrent Neural Networks (RNN)
* Long Short Term Memory (LSTM)
* Autoencoders
* GoogLeNet
* Overfeat
...


#### Symbolic variables in Theano
* Variable (C, Java, Python, etc.)
    * A segment of physical storage in RAM
    * Operations are based on value passing between variables
* Tensor (Theano)
    * A mathematical symbol
    * No physical storage in RAM to hold its value
    * Operations are actually building connections between tensors
* Shared variable (Theano)
    * Hybrid of variable and tensor
    * Tensor with physical storage in RAM to hold its value


In [ ]:
import theano
import theano.tensor as T

x = T.dvector(name='x')
def f(x):
    return x ** 2
y = f(x)

theano.printing.pydotprint(theano.function([x], y), '1.png')

![image](1.png)

_theano.function_ brings life to theano variables.

In [ ]:
import theano
import theano.tensor as T
import numpy as np

x = T.dvector(name='x')
def f(x):
    return x ** 2
y = f(x)

pow2 = theano.function(inputs=[x], outputs=y)

a = np.array([1,2,3], dtype=theano.config.floatX)
b = pow2(a)
print "a is {a}, b is {b}".format(a=a, b=b)

In [ ]:
import theano
import theano.tensor as T
import numpy as np

x = T.dvector('x')
y = x.sum()
grad = T.grad(cost=y, wrt=[x])
grad_func = theano.function(inputs=[x], outputs=grad)

a = np.array([1,2,3], dtype=theano.config.floatX)
print grad_func(a)

## Softmax Regression

### The Model

In the softmax regression setting, we are interested in multi-class classification. Suppose we have $m$ samples in the training set $\{(x^{(1)}, y^{(1)}),...,(x^{(m)}, y^{(m)})\}$, where $y^{(i)}\in \{1,2,...,k\}$ and $x^{(i)}\in R^{n}$.

Given a test sample $x$, we want to estimate the probability that $x$ belongs to class $j$, i.e., $p(y=j|x)$, for all possible $j$.

\begin{equation}
h_{W,b}(x) = 
\left[
  \begin{array}{c}
  p(y=1|x;W,b)\\
  p(y=2|x;W,b)\\
  ...\\
  p(y=k|x;W,b)\\
  \end{array}
\right]
=\frac{1}{\sum_{j=1}^{k}{e^{w_j^Tx+b_j}}}
\left[
  \begin{array}{c}
  e^{w_1^Tx+b_1}\\
  e^{w_2^Tx+b_2}\\
  ...\\
  e^{w_k^Tx+b_k}\\
  \end{array}
\right]
\end{equation}

When you implement softmax regression, it is usually convenient to represent $W$ as a $n$-by-$k$ matrix, so that
\begin{equation}
W = \left[\begin{array}{c}
w_1^T\\
...\\
w_k^T
\end{array}\right]
\end{equation}


### Defining a Loss Function
The loss of $h_{W, b}$ on the trainig set is
\begin{equation}
J(W, b) = -\frac{1}{m}\left[\sum_{i=1}^{m}\sum_{j=1}^{k}1\{y^{(i)}=j\}\log\frac{e^{w_j^Tx^{(i)}}}{\sum_{l=1}^{k}e^{w_l^Tx^{(i)}}}\right] + \frac{\lambda}{2}\sum_{i=1}^{k}\sum_{j=1}^{n}w_{ij}^2
\end{equation}

The second term is a weight decay term to disambiguate $W$ and $b$ that could yeild the least training error.


$J(W,b)$ is a convex function, and thus gradient descent will not run into a local optima problem.

### Learning the Model

Gradient descent

$$W \leftarrow W - \alpha \frac{\partial{J(W,b)}}{\partial{W}}$$

$$b \leftarrow b - \alpha \frac{\partial{J(W,b)}}{\partial{b}}$$

where $\alpha$ is the learning rate.

$$\frac{\partial{J(W,b)}}{\partial{w_j}} = -\frac{1}{m}\sum_{i=1}^{m}\left[x^{(i)}(1\{y^{(i)}\} - p(y^{(i)}=j|x^{(i)};W,b))\right] + \lambda w_j$$

$$\frac{\partial{J(W,b)}}{\partial{b}} = -\frac{1}{m}\sum_{i=1}^{m}\left(1\{y^{(i)}\} - p(y^{(i)}=j|x^{(i)};W,b)\right)$$

A more advanced option is the L-BFGS alogrithm, which also requires the gradient function as an input argument.


### Building the Model with Theano

#### Vectorising the model

Notations
* X: data matrix of size $n\times m$, where $n$ is the number of dimensions and m the number of instances. That is, each column stores an instance.
* W: weight matrix of size $k \times n$, where $k$ is the number of classes.
* b: bias vector of size $k\times 1$.

The posterior probability is
\begin{equation}
p = softmax(WX+b)
\end{equation}
where softmax(M) = M / M.sum(axis=0) and M is a arbitrary matrix.

#### Utilities for manipulating the paramters

In [1]:
import numpy as np

def get_size(shape):
    """
    count the number of elements in a ndarray with shape=shape
    """
    size = 1
    for i in shape:
        size *= i
    return size
    
def pack(param_list):
    """
    Args:
        param_list: list of ndarrays
    Returns:
        theta:  vector of shape (?,), flattened params
        shapes: list of tuples, with each tuple storing the shape of each ndarray in param_list
    """
    shapes = []
    theta=None
    for p in param_list:
        size = p.size
        p2 = p.reshape((size, )) 
        if theta is None:
            theta = p2
        else:
            theta = np.hstack((theta, p2))
        shapes += [p.shape] 
    return theta, shapes
        
def unpack(theta, shapes):
    """
    Args:
        theta:  vector of shape (?,), flattened params
        shapes: list of tuples, with each tuple storing the shape of each ndarray in param_list      
    Returns:
        param_list: list of ndarrays
    """
    i = 0
    params = []
    for shape in shapes:
        size = get_size(shape)
        x = theta[i:i+size]
        params += [x.reshape(shape)]
        i += size
    return params

#### Defining the softmax regression class

In [2]:
#coding=utf-8
import theano
import theano.tensor as T
import numpy as np
import scipy as sp
import gzip
import cPickle
import matplotlib.pyplot as plt

#from param_util import pack, unpack

# for debugging
theano.config.optimizer="fast_run"
theano.config.exception_verbosity="high"

class SoftmaxRegression(object):
    def __init__(self, n_in, n_out, L2_reg_coef, max_iter=100):
        self.n_in = n_in
        self.n_out = n_out
        self.L2_reg_coef = L2_reg_coef
        self.max_iter = max_iter
        
        self.W = theano.shared(
            value=0.005 * np.random.randn(self.n_out, self.n_in),
            name='W',
            borrow=True)
        self.b = theano.shared(
            value=np.zeros((self.n_out,), dtype=theano.config.floatX),
            name='b',
            borrow=True)
        self.params = [self.W, self.b]
        
        # sample matrix, each column stores a sample
        X = T.dmatrix('X')
        # label
        y = T.lvector('y')
        # predict
        linear_input = (T.dot(self.W, X).transpose() + self.b).transpose()
        p_y_given_x = self.__softmax__(linear_input)
        pred = T.argmax(p_y_given_x, axis=0)
        # NLL: negative log-likelihood
        nll = -T.mean(T.log(p_y_given_x[y, T.arange(0, y.shape[0])]))
        # cost (loss)
        cost = nll + self.L2_reg_coef * (self.W**2).sum()
        # error rate
        error = 1.0*T.sum(T.neq(pred, y))/X.shape[1]
        
        self.test_model = theano.function(
            inputs=[X, y],
            outputs=error)
        self.run_model = theano.function(
            inputs=[X],
            outputs=pred)
            
        # compute gradient
        grads = T.grad(cost, self.params)
        params = [p.type() for p in self.params]
        self.grad_func = theano.function(
            inputs=params + [X, y],
            outputs=grads,
            givens=[(s, p) for s, p in zip(self.params, params)])
        self.cost_func = theano.function(
            inputs=params + [X, y],
            outputs=cost,
            givens=[(s, p) for s, p in zip(self.params, params)])   
        
    def fit(self, X, y):
        init_theta, shapes = pack([self.W.get_value(), self.b.get_value()])
        opt_theta, opt_cost, d = sp.optimize.fmin_l_bfgs_b(
            func=self.__cost_and_grad__,
            x0=init_theta,
            fprime=None,
            args=(shapes, X, y),
            maxiter=self.max_iter,
            iprint=1)
        opt_param_list = unpack(opt_theta, shapes)
        for shared_param, opt_param in zip(self.params, opt_param_list):
             shared_param.set_value(opt_param)      

    def transform(self, X):
        return self.run_model(X)
    
    def evaluate(self, X, y):
        return self.test_model(X, y)
        
    def __cost_and_grad__(self, theta, *args):
        shapes = args[0]
        X = args[1]
        y = args[2]
        param_list = unpack(theta, shapes)
        cost =  self.cost_func(*(param_list + [X, y]))
        grad = self.grad_func(*(param_list + [X, y]))
        grad, _ = pack(grad)
        return cost, grad        
    
    def __softmax__(self, M):
        """
        normalise along the vertical axis
        """
        e_M = T.exp(M - M.max(axis=0, keepdims=True))
        return e_M / e_M.sum(axis=0, keepdims=True)

#### Evaluating the model

#### Dataset: MNIST Dataset of Handwritten Digits

* Gray-scale image of size $28\times 28$ with value ranging from [0, 1].
* 50,000 training samples, 10,000 validation samples and 10,000 testing samples.

![image](mnist.png)

In [3]:
if __name__ == '__main__':
    plt.close('all')
    np.random.seed(0)
    
    f = gzip.open('data/mnist.pkl.gz')
    train_set, valid_set, test_set = cPickle.load(f)
    
    # train model
    train_X = train_set[0].transpose()
    train_y = train_set[1]
    valid_X = valid_set[0].transpose()
    valid_y = valid_set[1]
    
    train_X = np.hstack((train_X, valid_X))
    train_y = np.hstack((train_y, valid_y))
    
    train_X = train_X[:, 0:60001:5]
    train_y = train_y[0:60001:5]
    
    n_in = train_X.shape[0]
    n_out = np.unique(train_y).shape[0]
    print "{n} training samples of dim {d}".format(n=train_X.shape[1], d=n_in)
    
    softmax_regression = SoftmaxRegression(n_in, n_out, L2_reg_coef=1e-4, max_iter=100)
    softmax_regression.fit(train_X, train_y)
    
    # test model
    test_X = test_set[0].transpose()
    test_y = test_set[1]
    error = softmax_regression.evaluate(test_X, test_y)
    print 'error rate on test set is {e}%, accuracy is {a}%'.format(e=error*100, a=100-100*error)
    # baseline method
    pred_baseline = np.random.randint(
        low = np.min(train_y),
        high = np.max(train_y)+1,
        size=valid_y.shape)
    error_baseline = 1.0 * (pred_baseline != valid_y).sum() / valid_X.shape[1]
    print 'baseline: error rate on test set is {e}%, accurary is {a}%'.format(e=error_baseline*100, a=100-100*error_baseline)


12000 training samples of dim 784
error rate on test set is 8.5%, accuracy is 91.5%
baseline: error rate on test set is 90.18%, accurary is 9.82%


#### What are the "standard digits" in the eyes of Softmax Regression?

Find input $x$ that maximally activates the $k$-th output unit and has limited $L_2$ norm: 
$$
\max_{x}h_k(x)\\
s.t. \Vert x \Vert^2 \le 1
$$
This problem can be approximated by a non-constrained minimisation problem:
$$
\min_{x}\left[-h_k(x) + \lambda \max(\Vert x \Vert^2 - 1, 0)\right]
$$
where $\lambda$ is a fairly large number.

In [4]:
# visulisation
plt.close('all')
def rescale(x):
    # Remove DC (mean of images)
    x = x - x.mean()
    # Truncate to +/-3 standard deviations and scale to -1 to 1
    pstd = 3 * x.std()
    x = np.maximum(np.minimum(x, pstd), -pstd) / pstd
    # Rescale from [-1,1] to [0.1,0.9]
    x = (x + 1) * 0.4 + 0.1;
    return x 
def softmax(M):
    e_M = T.exp(M - M.max(axis=0, keepdims=True))
    return e_M / e_M.sum(axis=0, keepdims=True)
W = softmax_regression.W.get_value()
b = softmax_regression.b.get_value()
fig = plt.figure(0)
for i in np.arange(0, 10):
    x = T.dvector('x')
    x0 = np.random.rand(28*28)
    x0 = x0 / np.sqrt((x0**2).sum())
    p_y_given_x = softmax(T.dot(W, x) + b)
    # cost
    cost = -T.log(p_y_given_x[i])  + 1e2 * T.max([0, (x**2).sum()-1])**2
    cost_func = theano.function([x], cost)
    grad = T.grad(cost, x)
    grad_func = theano.function([x], grad)
    proba_func = theano.function([x], p_y_given_x)
    def cost_and_grad(x):
        return cost_func(x), grad_func(x)
    opt_x, opt_cost, d = sp.optimize.fmin_l_bfgs_b(
        func=cost_and_grad,
        x0=x0,
        maxiter=100,
        iprint=0)
    #print opt_x
    print "||x||^2 is {v:.2f}, proab is {p:.4f}".format(v=(opt_x**2).sum(), p=proba_func(opt_x)[i])
    opt_x_rescaled = rescale(opt_x)
    img = opt_x_rescaled.reshape((28,28))
    ax = fig.add_subplot(2, 5,i+1)
    ax.imshow(img, cmap='gray', interpolation='none')
    ax.set_axis_off()
plt.show()
fig.savefig('visual_softmax_regression.png', format='png')

||x||^2 is 1.00, proab is 0.9381
||x||^2 is 1.00, proab is 0.9924
||x||^2 is 1.00, proab is 0.9972
||x||^2 is 1.00, proab is 0.9806
||x||^2 is 1.00, proab is 0.9873
||x||^2 is 1.00, proab is 0.9997
||x||^2 is 1.00, proab is 0.9800
||x||^2 is 1.00, proab is 0.9986
||x||^2 is 1.01, proab is 0.6853
||x||^2 is 1.00, proab is 0.9644


/Users/taiqing/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/matplotlib/image.py:649: UserWarning: The backend (<class 'matplotlib.backends.backend_macosx.RendererMac'>) does not support interpolation='none'. The image will be interpolated with 'nearest` mode.
  "mode." % renderer.__class__)


![image](visual_softmax_regression_digits.png)

## Highlights of DNN

## Neural Network

### The Model

#### Neuron: Building block of neural network

![image](neuron_brain.jpg)
![image](neuron.png)

$$h_{W,b}(x) = f(W^Tx+b) = f(\sum_{i=1}^{3}W_ix_i + b)$$
, where $f: \mathcal{R}\rightarrow\mathcal{R}$ is called the activation function. Common choices for $f$ include the sigmoid function and the tanh function. We use the sigmoid function in this talk.

sigmoid function:
$$f(z) = \frac{1}{1 + \exp(-z)}$$

tanh function:
$$f(z) = \frac{e^z - e^{-z}}{e^z + e^{-z}}$$

![image](sigmoid.png) ![image](tanh.png)

#### Neural network model

![image](nn.png)

##### Notations
* $m$: the number of samples
* $n_l$: the number of layers, including the input, hidden and output layers
* $s_l$: the number of units in layer $L_l$, excluding the bias unit
* $W^{(l)}$: the weight matrix connecting layer $L_l$ and layer $L_{l+1}$
* $W_{ji}^{(l)}$: the weight connecting unit $i$ in layer $L_l$ and unit $j$ in layer $L_{l+1}$
* $b^{(l)}$: the bias vector connecting the bias unit in layer $L_l$ to the units in layer $L_{l+1}$
* $z^{(l)}_j$: the linear input of the unit $j$ in layer $L_l$
* $a^{(l)}_j$: the activation of the unit $j$ in layer $L_l$

![image](simple_nn.png)

$$a_1^{(2)} = f(W_{11}^{(1)}x_1 + W_{12}^{(1)}x_1 + W_{13}^{(1)}x_1 + b_1^{(1)})$$

$$a_2^{(2)} = f(W_{21}^{(1)}x_1 + W_{22}^{(1)}x_1 + W_{23}^{(1)}x_1 + b_1^{(2)})$$

$$a_1^{(3)} = f(W_{31}^{(1)}x_1 + W_{32}^{(1)}x_1 + W_{33}^{(1)}x_1 + b_1^{(3)})$$

$$h_{W,b}(x) = a_1^{(3)} = f(W_{11}^{(2)}a_1 + W_{12}^{(2)}a_1 + W_{13}^{(2)}a_1 + b_1^{(2)})$$

If we extend the activation function $f(\cdot)$ to apply to vectors in an element-wise fashion (i.e., $f([z_1, z_2, z_3]) = [f(z_1), f(z_2), f(z_3)])$, we can rewrite the equations above more compactly as

$$z^{(2)} = W^{(1)}x + b^{(1)}$$

$$a^{(2)} = f(z^{(2)})$$

$$z^{(3)} = W^{(2)}a^{(2)} + b^{(2)}$$

$$h_{W,b}(x) = a^{(3)} = f(z^{(3)})$$

### Forward Propagation - RUNNING

Set $a^{(1)} = x$ to denote the values from the input layer. Given layer $L_l$'s activations $a^{(l)}$, we can compute layer $L_{l+1}$'s activations $a^{(l+1)}$ as

$$z^{(l+1)} = W^{(l)}a^{(l)} + b^{(l)}$$

$$a^{(l+1)} = f(z^{(l+1)})$$

### Backward Propagation - TRAINING

## Autoencoder

### The Model

Suppose we have only a set of unlabeled training examples $\{x^{(1)},...,x^{(m)}\}$, where $x^{(i)}\in R^n$.
An autoencoder neural network is an unsupervised learning algorithm that applies backpropagation, setting the target values to be equal to the inputs, i.e., $y^{(i)}=x^{(i)}$.

![image](autoencoder.png)

The autoencoder tries to learn an identity function $h_{W,b}\approx x$.
* The identity function seems a particularly trivial function to be trying to learn;
* By placing constraints on the network, such as by limiting the number of hidden units, we can discover interesting structure about the data.
* As a concrete example, suppose the inputs $x$ are the pixel intensity values from a $10\times 10$ image ($100$ pixels) so $n=100$, and there are $s_2=50$ hidden units in layer $L_2$. 
Since there are only $50$ hidden units, the network is forced to learn a compressed representation of the input.
* If the inputs are completely random, i.e., each dimension comes from an independent distribution, the autoencoding task would be very difficult.

Notations
* $m$: the number of samples
* $n_l$: the number of layers, including the input, hidden and output layers
* $s_l$: the number of units in layer $L_l$, excluding the bias unit
* $W^{(l)}$: the weight matrix connecting layer $L_l$ and layer $L_{l+1}$
* $W_{ji}^{(l)}$: the weight connecting unit $i$ in layer $L_l$ and unit $j$ in layer $L_{l+1}$
* $b^{(l)}$: the bias vector connecting the bias unit in layer $L_l$ to the units in layer $L_{l+1}$
* $z^{(l)}_j$: the linear input of the unit $j$ in layer $L_l$
* $a^{(l)}_j$: the activation of the unit $j$ in layer $L_l$

Foward propagation
$$z^{(2)} = W^{(1)}x + b^{(1)}$$

$$a^{(2)} = \text{sigmoid}(z^{(2)})$$

$$z^{(3)} = W^{(2)}a^{(2)} + b^{(2)}$$

$$a^{(3)} = \text{sigmoid}(z^{(3)})$$

$$h_{W,b}(x) = a^{(3)}(x)$$

### Defining a Loss Function

For a single training sample $x$, the loss function is defined as
\begin{equation}
J(W,b; x) = \frac{1}{2}\Vert h_{W,b}(x)-x\Vert^2
\end{equation}
Given a trainig set of $m$ samoples, we define the loss function as
\begin{equation}
\begin{aligned}
J(W,b) &= 
\left[\frac{1}{m}\sum_{i=1}^{m}J(W,b; x^{(i)})\right]
+\lambda \sum_{l=1}^{n_{l-1}}\sum_{i=1}^{s_l}\sum_{j=1}^{s_{l+1}}\left(W_{ji}^{(l)}\right)^2\\
&=\left[\frac{1}{m}\sum_{i=1}^{m}\left(\frac{1}{2}\Vert h_{W,}(x^{(i)})-x^{(i)}\Vert^2\right)\right]
+\lambda \sum_{l=1}^{n_{l-1}}\sum_{i=1}^{s_l}\sum_{j=1}^{s_{l+1}}\left(W_{ji}^{(l)}\right)^2
\end{aligned}
\end{equation}

The second term is a weight decay term.

### Sparsity Constraint

We would like to constrain the neurons to be active only for a subset of patterns.
* $a^{(2)}_j(x)$ denotes the activiation of hidden unit $j$ when the network is given a specific input $x$.
* The average activation of hidden unit $j$ over the training set is
\begin{equation}
\hat{\rho}_j = \frac{1}{m}\sum_{i=1}^{m}\left[a^{(2)}_j(x)\right]
\end{equation}
* We would like to enforce the constraint
$$\hat{\rho}_j=\rho$$
where $\rho$ is a sparsity parameter, typically a small value close to zero.
* A penalty term that penalises $\hat{\rho}_j$ deviating significantly from $\rho$
\begin{equation}
\sum_{j=1}^{s_2}KL(\rho||\hat{\rho}_j) = \sum_{j=1}^{s_2}\left[\rho\log\frac{\rho}{\hat{\rho}_j} + (1-\rho)\log\frac{1-\rho}{1-\hat{\rho}_j}\right]
\end{equation}
    * $KL(\rho||\hat{\rho}_j)$ has the property that $KL(\rho||\hat{\rho}_j) = 0$ if $\hat{\rho}_j=\rho$, and otherwise it increases monotonically as $\hat{\rho}_j$ diverges from $\rho$.
    * In the figure below, we set $\rho = 0.2$.

![image](kl.png)

Our overall loss function for sparse autoencoder is
\begin{equation}
\begin{aligned}
J(W,b) &= 
\left[\frac{1}{m}\sum_{i=1}^{m}J(W,b; x^{(i)})\right]
+\lambda \sum_{l=1}^{n_{l-1}}\sum_{i=1}^{s_l}\sum_{j=1}^{s_{l+1}}\left(W_{ji}^{(l)}\right)^2 + \beta\sum_{j=1}^{s_2}KL(\rho||\hat{\rho}_j)\\
&=\left[\frac{1}{m}\sum_{i=1}^{m}\left(\frac{1}{2}\Vert h_{W,b}(x^{(i)})-x^{(i)}\Vert^2\right)\right]
+\lambda \sum_{l=1}^{n_{l-1}}\sum_{i=1}^{s_l}\sum_{j=1}^{s_{l+1}}\left(W_{ji}^{(l)}\right)^2 + \beta \sum_{j=1}^{s_2}\left[\rho\log\frac{\rho}{\hat{\rho}_j} + (1-\rho)\log\frac{1-\rho}{1-\hat{\rho}_j}\right]
\end{aligned}
\end{equation}

### Building the Model with Theano

#### Utilities for paramter manipulations

In [6]:
#coding=utf-8
import numpy as np

def get_size(shape):
    """
    count the number of elements in a ndarray with shape=shape
    """
    size = 1
    for i in shape:
        size *= i
    return size
    
def pack(param_list):
    """
    Args:
        param_list: list of ndarrays
    Returns:
        theta:  vector of shape (?,), flattened params
        shapes: list of tuples, with each tuple storing the shape of each ndarray in param_list
    """
    shapes = []
    theta=None
    for p in param_list:
        size = p.size
        p2 = p.reshape((size, )) 
        if theta is None:
            theta = p2
        else:
            theta = np.hstack((theta, p2))
        shapes += [p.shape] 
    return theta, shapes
        
def unpack(theta, shapes):
    """
    Args:
        theta:  vector of shape (?,), flattened params
        shapes: list of tuples, with each tuple storing the shape of each ndarray in param_list      
    Returns:
        param_list: list of ndarrays
    """
    i = 0
    params = []
    for shape in shapes:
        size = get_size(shape)
        x = theta[i:i+size]
        params += [x.reshape(shape)]
        i += size
    return params

#### Defining a single layer

In [7]:
#coding=utf-8
import theano
import theano.tensor as T
import numpy as np

class Layer(object):
    def __init__(self, n_in, n_out, name=''):
        r = -np.sqrt(6.0 / (n_in + n_out + 1))
        rand_W = 2 * r * np.random.rand(n_out, n_in) - r
        self.W = theano.shared(
            value=rand_W,
            name='W_'+name,
            borrow=True)
        self.b = theano.shared(
            value=np.zeros((n_out,), dtype=theano.config.floatX),
            name='b_'+name,
            borrow=True)
        self.params = [self.W, self.b]
        self.L2_sqr = (self.W ** 2).sum()
        
    def forward(self, X):
        """
        X: matrix tensor, each column stores a sample
        """
        linear_input = (T.dot(self.W, X).transpose() + self.b).transpose()
        activation = T.nnet.sigmoid(linear_input)
        return activation

#### Defining a AutoEncoder class

In [9]:
#coding=utf-8
import theano
import theano.tensor as T
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import gzip
import cPickle

#from param_util import pack, unpack
#from layer import Layer

# for debugging
theano.config.optimizer='fast_run'#'fast_compile'
theano.config.exception_verbosity="high"

class AutoEncoder(object):
    def __init__(self, n_in, n_hid, 
                 L2_reg_coef, sparse_reg_coef, sparse_rho, 
                 max_iter=400):
        self.n_in = n_in
        self.n_hid = n_hid
        self.L2_reg_coef = L2_reg_coef
        self.sparse_reg_coef = sparse_reg_coef
        self.sparse_rho = sparse_rho
        self.max_iter = max_iter 
        
        self.hidden_layer = Layer(self.n_in, self.n_hid, 'hidden')
        self.output_layer = Layer(self.n_hid, self.n_in, 'output')
        self.params = self.hidden_layer.params + self.output_layer.params
        
        X = T.dmatrix(name='X') # each column stores a sample
        m = X.shape[1] # sample count
        activation = self.hidden_layer.forward(X)
        output = self.output_layer.forward(activation)
        error = ((output - X) ** 2).sum() / (2.0 * m)
        L2_reg = self.hidden_layer.L2_sqr + self.output_layer.L2_sqr
        rho = activation.mean(axis=1)
        kl = (self.sparse_rho * T.log(self.sparse_rho/rho) + 
              (1-self.sparse_rho) * T.log((1-self.sparse_rho)/(1-rho))).sum()
        cost = error + self.L2_reg_coef * 0.5 * L2_reg + self.sparse_reg_coef * kl
        grads = T.grad(cost, self.params)
        params = [p.type() for p in self.params]
        self.cost_func = theano.function(
            inputs=params + [X],
            outputs=cost,
            givens=[(s, p) for s, p in zip(self.params, params)])
        self.grad_func = theano.function(
            inputs=params + [X],
            outputs=grads,
            givens=[(s, p) for s, p in zip(self.params, params)])
        self.run_model = theano.function(
            inputs=[X],
            outputs=activation)
    
    def forward(self, X):
        """
        X: matrix tensor, each column stores a sample
        """
        return self.hidden_layer.forward(X)
            
    def fit(self, X):
        """
        X: np.ndarray, each column stores a sample
        """
        init_theta, shapes = pack([
            self.hidden_layer.W.get_value(), self.hidden_layer.b.get_value(), 
            self.output_layer.W.get_value(), self.output_layer.b.get_value()])
        opt_theta, opt_cost, d = sp.optimize.fmin_l_bfgs_b(
            func=self.cost_and_grad,
            x0=init_theta,
            fprime=None,
            args=(shapes, X),
            maxiter=self.max_iter,
            iprint=1)
        opt_param_list = unpack(opt_theta, shapes)
        for shared_param, opt_param in zip(self.params, opt_param_list):
             shared_param.set_value(opt_param)
             
    def transform(self, X):
        """
        X: np.ndarray, each column stores a sample
        """
        return self.run_model(X)

    def cost_and_grad(self, theta, *args):
        """
        compute the cost and gradient at theta in the parameter space
        args should be (shapes, X)
        X: each column stores a sample
        """
        shapes = args[0]
        X = args[1]
        param_list = unpack(theta, shapes)
        cost =  self.cost_func(*(param_list+[X]))
        grad = self.grad_func(*(param_list+[X]))
        grad, _ = pack(grad)
        return cost, grad

#### Training the model

In [10]:
if __name__ == '__main__':
    plt.close('all')
    np.random.seed(0)
    
    #X = cPickle.load(open('data/patches', 'r'))
    f = gzip.open('data/mnist.pkl.gz')
    train_set, valid_set, test_set = cPickle.load(f)
    train_X = train_set[0].transpose()
    X = train_X[:, 0:50001:5]
    print '{m} samples of dimension {d}'.format(m=X.shape[1], d=X.shape[0])
    auto_encoder = AutoEncoder(
        n_in=X.shape[0],
        n_hid = 200,
        L2_reg_coef=3e-3,
        sparse_reg_coef=3.0,
        sparse_rho=0.1,
        max_iter=200)
    auto_encoder.fit(X)

10000 samples of dimension 784


### Visualising a Trained Autoencoder

Given a specific input $x$, the activation of hidden unit $i$ is
$$a^{(2)}_i(x) = \text{sigmoid}(\sum_{j=1}^{s_1}W_{ij}^{(1)}x_j + b_i)$$

Find the input $x^*$ that maximises $a^{(2)}_i$, i.e., causes hidden unit $i$ to be maximally activated.

Considering
* Sigmoid function is a monotonically increasing function;
* The input should be constrained to have a limited magnitude,

we have
\begin{equation}
x^* = \arg\max_{x}{\sum_{j=1}^{s_1}W_{ij}^{(1)}x_j + b_i}\\
s.t. \Vert x \Vert^2 \le 1
\end{equation}

** Lemma 1** The optimal $x^*$ has identity norm, that is
$$\Vert x^* \Vert^2 = 1$$

Solve for $x^*$ by lagrange multiplier:
\begin{equation}
F(x, \lambda) = {\sum_{j=1}^{s_1}W_{ij}^{(1)}x_j + b_i} + \lambda \left(\sum_{j=1}^{s_1}x_j^2-1\right)
\end{equation}

Setting
\begin{equation}
\frac{\partial F(x, \lambda)}{\partial x_j} = W_{ij}^{(1)}x_j + 2\lambda x_j = 0
\end{equation}

\begin{equation}
\frac{\partial F(x, \lambda)}{\partial \lambda} = \sum_{j=1}^{s_1}x_j^2 - 1 = 0
\end{equation}

By solving the above equations, we have
\begin{equation}
x_j^* = \frac{W_{ij}^{(1)}}{\sqrt{\sum_{j=1}^{s_1}\left(W_{ij}^{(1)}\right)^2}}
\end{equation}



In [12]:
    fig = plt.figure(0)
    n = 8#int(np.floor(np.sqrt(auto_encoder.n_hid)))
    for i in np.arange(n**2):
        ax = fig.add_subplot(n, n, i+1)
        w = auto_encoder.hidden_layer.W.get_value()[i, :]
        w = w / np.sqrt((w ** 2).sum())
        w = (w - w.min()) / (w.max()-w.min())
        d = np.floor(np.sqrt(w.shape[0]))
        ax.imshow(w.reshape((d,d)), cmap='gray', interpolation='none')
        ax.set_axis_off()
    plt.show()
    fig.savefig('visual_ae.png', format='png')
    
    # visulisation the first hidden layer
    def rescale(x):
        # Remove DC (mean of images)
        x = x - x.mean()
        # Truncate to +/-3 standard deviations and scale to -1 to 1
        pstd = 3 * x.std()
        x = np.maximum(np.minimum(x, pstd), -pstd) / pstd
        # Rescale from [-1,1] to [0.1,0.9]
        x = (x + 1) * 0.4 + 0.1;
        return x
    fig = plt.figure(0)
    k = 0
    for i in range(0, 25):#for i in np.random.randint(0, dnn.ae_layers[0].n_hid, 64):
        x = T.dvector('x')
        x0 = np.random.rand(28*28)
        x0 = x0 / np.sqrt((x0**2).sum())
        a = auto_encoder.forward(x)
        # cost
        cost = -a[i] + 1e2 * T.max([0, (x**2).sum()-1])**2
        grad = T.grad(cost, x)
        cost_func = theano.function([x], cost)
        grad = T.grad(cost, x)
        grad_func = theano.function([x], grad)
        act_func = theano.function([x], a)
        def cost_and_grad(x):
            return cost_func(x), grad_func(x)
        opt_x, opt_cost, d = sp.optimize.fmin_l_bfgs_b(
            func=cost_and_grad,
            x0=x0,
            maxiter=1000,
            iprint=0)
        opt_x_rescaled = rescale(opt_x)
        img = opt_x_rescaled.reshape((28,28))
        k = k + 1
        ax = fig.add_subplot(5, 5, k)
        ax.imshow(img, cmap='gray', interpolation='none')
        ax.set_axis_off()
        
        w = auto_encoder.hidden_layer.W.get_value()[i,:]
        w = w / np.sqrt((w ** 2).sum())
        print ("||x||^2 is {v:.2f}, activation is {a:.4f}, {t:.4f} (optimal)"
            .format(v=(opt_x**2).sum(), a=act_func(opt_x)[i], t=act_func(w)[i]))
    plt.show()
    fig.savefig('visual_ae2.png', format='png')

![image](visual_autoencoder.png)

## Building Deep Networks for Classification

### Advantages of Deep Networks
* Deep network can compactly represent a significantly larger set of functions than shallow networks.
    * For a network with $n$ inputs, $l$ hidden layers and sigmoid activation, the bound of the complexity of the function implemented by a feedforward neural network is $2^{l-1}$. [M. Bianchini and F. Scarselli, On the complexity of shallow and deep neural network classifiers, ESANN14]
* By using a deep network, one can start to learn part-whole decompositions in the case of images.
    * The first layer might learn to detect edges,
    * The second layer might learn to group edges to detect longer contours,
    * The thirt layer might learn to detect simple parts of objects, and so on.
    ![image](part_whole.png)
* Cortical computations in the brain also have multiple layers of processing.
    * Visual images are processed in multiple stages by the brain, by cortical area "V1", followed by cortical area "V2", and so on.
    ![image](cortical.png)

### Difficulties of Training Deep Architectures

The main learning algorithmthat researchers were using was to randomly initialise the weights of the a deep network, and then train it using a labeled training set $\{(x^{(i)}, y^{(i)}),i=1...m\}$ using a supervised learning objective. However, this usually did not work well for deep networks.

#### Availability of data
* Given the high degree of expressive power of deep networks, training on insufficient data would result in overfitting.
* Labeled data is often scarce and sometimes expensive.
* For many problems, it is difficult to get enough samples to fit the parameters of a complex model.

#### Local optima
* Training a deep network using supervised learning involves solving a highly non-convex optimisation problem, which is rife with bad local optima.
* Trainig with gradient descent (or methods like conjugate gradient and L-BFGS) can easily get trapped in bad local optima.

#### Diffusion of gradients
* The gradients that are propagated backwards rapidly diminish in magnitude as the depth of the network increases.
* When using gradient descent, the weights of the earlier layers change slowly and the earlier layers fail to learn much.
* Training a deep network ends up giving similar performance to training a shallow network.


### Greedy Layer-wise Training
The main idea is 
* first to train the hidden layers of the network one at a time in an unsurpervised manner,
* then to fine-tune the whole network with labeled data in a supervised manner.

Advantages of greedy layer-wise training include
#### Availabilty of data
* Enable to take advantage of the cheap and plentiful unlabeled data

#### Better local optima
* The weights are now starting at a better location in parameter space than if they had been randomly initialised.

### Building the Model with Theano

This section will implement a deep architecture with stacked autoencoders and a softmax regression as the output layer. 

#### Utilities for manipluating parameters

In [ ]:
#coding=utf-8
import numpy as np

def get_size(shape):
    """
    count the number of elements in a ndarray with shape=shape
    """
    size = 1
    for i in shape:
        size *= i
    return size
    
def pack(param_list):
    """
    Args:
        param_list: list of ndarrays
    Returns:
        theta:  vector of shape (?,), flattened params
        shapes: list of tuples, with each tuple storing the shape of each ndarray in param_list
    """
    shapes = []
    theta=None
    for p in param_list:
        size = p.size
        p2 = p.reshape((size, )) 
        if theta is None:
            theta = p2
        else:
            theta = np.hstack((theta, p2))
        shapes += [p.shape] 
    return theta, shapes
        
def unpack(theta, shapes):
    """
    Args:
        theta:  vector of shape (?,), flattened params
        shapes: list of tuples, with each tuple storing the shape of each ndarray in param_list      
    Returns:
        param_list: list of ndarrays
    """
    i = 0
    params = []
    for shape in shapes:
        size = get_size(shape)
        x = theta[i:i+size]
        params += [x.reshape(shape)]
        i += size
    return params

#### A single layer

In [ ]:
#coding=utf-8
import theano
import theano.tensor as T
import numpy as np

class Layer(object):
    def __init__(self, n_in, n_out, name=''):
        r = -np.sqrt(6.0 / (n_in + n_out + 1))
        rand_W = 2 * r * np.random.rand(n_out, n_in) - r
        self.W = theano.shared(
            value=rand_W,
            name='W_'+name,
            borrow=True)
        self.b = theano.shared(
            value=np.zeros((n_out,), dtype=theano.config.floatX),
            name='b_'+name,
            borrow=True)
        self.params = [self.W, self.b]
        self.L2_sqr = (self.W ** 2).sum()
        
    def forward(self, X):
        """
        X: matrix tensor, each column stores a sample
        """
        linear_input = (T.dot(self.W, X).transpose() + self.b).transpose()
        activation = T.nnet.sigmoid(linear_input)
        return activation

#### AutoEncoder class

In [ ]:
#coding=utf-8
import theano
import theano.tensor as T
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import gzip
import cPickle

#from param_util import pack, unpack
#from layer import Layer

# for debugging
theano.config.optimizer='fast_run'#'fast_compile'
theano.config.exception_verbosity="high"

class AutoEncoder(object):
    def __init__(self, n_in, n_hid, 
                 L2_reg_coef, sparse_reg_coef, sparse_rho, 
                 max_iter=400):
        self.n_in = n_in
        self.n_hid = n_hid
        self.L2_reg_coef = L2_reg_coef
        self.sparse_reg_coef = sparse_reg_coef
        self.sparse_rho = sparse_rho
        self.max_iter = max_iter 
        
        self.hidden_layer = Layer(self.n_in, self.n_hid, 'hidden')
        self.output_layer = Layer(self.n_hid, self.n_in, 'output')
        self.params = self.hidden_layer.params + self.output_layer.params
        
        X = T.dmatrix(name='X') # each column stores a sample
        m = X.shape[1] # sample count
        activation = self.hidden_layer.forward(X)
        output = self.output_layer.forward(activation)
        error = ((output - X) ** 2).sum() / (2.0 * m)
        L2_reg = self.hidden_layer.L2_sqr + self.output_layer.L2_sqr
        rho = activation.mean(axis=1)
        kl = (self.sparse_rho * T.log(self.sparse_rho/rho) + 
              (1-self.sparse_rho) * T.log((1-self.sparse_rho)/(1-rho))).sum()
        cost = error + self.L2_reg_coef * 0.5 * L2_reg + self.sparse_reg_coef * kl
        grads = T.grad(cost, self.params)
        params = [p.type() for p in self.params]
        self.cost_func = theano.function(
            inputs=params + [X],
            outputs=cost,
            givens=[(s, p) for s, p in zip(self.params, params)])
        self.grad_func = theano.function(
            inputs=params + [X],
            outputs=grads,
            givens=[(s, p) for s, p in zip(self.params, params)])
        self.run_model = theano.function(
            inputs=[X],
            outputs=activation)
    
    def forward(self, X):
        """
        X: matrix tensor, each column stores a sample
        """
        return self.hidden_layer.forward(X)
            
    def fit(self, X):
        """
        X: np.ndarray, each column stores a sample
        """
        init_theta, shapes = pack([
            self.hidden_layer.W.get_value(), self.hidden_layer.b.get_value(), 
            self.output_layer.W.get_value(), self.output_layer.b.get_value()])
        opt_theta, opt_cost, d = sp.optimize.fmin_l_bfgs_b(
            func=self.cost_and_grad,
            x0=init_theta,
            fprime=None,
            args=(shapes, X),
            maxiter=self.max_iter,
            iprint=1)
        opt_param_list = unpack(opt_theta, shapes)
        for shared_param, opt_param in zip(self.params, opt_param_list):
             shared_param.set_value(opt_param)
             
    def transform(self, X):
        """
        X: np.ndarray, each column stores a sample
        """
        return self.run_model(X)

    def cost_and_grad(self, theta, *args):
        """
        compute the cost and gradient at theta in the parameter space
        args should be (shapes, X)
        X: each column stores a sample
        """
        shapes = args[0]
        X = args[1]
        param_list = unpack(theta, shapes)
        cost =  self.cost_func(*(param_list+[X]))
        grad = self.grad_func(*(param_list+[X]))
        grad, _ = pack(grad)
        return cost, grad

#### SoftmaxRegression class

In [ ]:
#coding=utf-8
import theano
import theano.tensor as T
import numpy as np
import scipy as sp
import gzip
import cPickle

from param_util import pack, unpack

# for debugging
theano.config.optimizer="fast_run"
theano.config.exception_verbosity="high"

class SoftmaxRegression(object):
    def __init__(self, n_in, n_out, L2_reg_coef, max_iter=100):
        self.n_in = n_in
        self.n_out = n_out
        self.L2_reg_coef = L2_reg_coef
        self.max_iter = max_iter
        
        self.W = theano.shared(
            value=0.005 * np.random.randn(self.n_out, self.n_in),
            name='W',
            borrow=True)
        self.b = theano.shared(
            value=np.zeros((self.n_out, ), dtype=theano.config.floatX),
            name='b',
            borrow=True)
        self.params = [self.W, self.b]
        
        # sample matrix, each column stores a sample
        X = T.dmatrix('X')
        # label
        y = T.lvector('y')
        # predict
        pred, p_y_given_x = self.predict(X)
        # cost
        cost = self.calc_cost(p_y_given_x, y)
        # error rate
        error = 1.0*T.sum(T.neq(pred, y))/X.shape[1]
        
        self.test_model = theano.function(
            inputs=[X, y],
            outputs=error)
        self.run_model = theano.function(
            inputs=[X],
            outputs=pred)
            
        # compute gradient
        grads = T.grad(cost, self.params)
        params = [p.type() for p in self.params]
        self.grad_func = theano.function(
            inputs=params + [X, y],
            outputs=grads,
            givens=[(s, p) for s, p in zip(self.params, params)])
        self.cost_func = theano.function(
            inputs=params + [X, y],
            outputs=cost,
            givens=[(s, p) for s, p in zip(self.params, params)])
    
    def calc_cost(self, p_y_given_x, y):
        """
        p_y_given_x: matrix tensor, output by self.predict
        y: vector tensor, labels
        """
        # NLL: negative log-likelihood
        nll = -T.mean(T.log(p_y_given_x[y, T.arange(0, y.shape[0])]))
        cost = nll + self.L2_reg_coef * (self.W**2).sum()
        return cost
    
    def predict(self, X):
        """
        X: matrix tensor, each column stores a sample
        """
        linear_input = (T.dot(self.W, X).transpose() + self.b).transpose()
        p_y_given_x = self.softmax(linear_input)
        pred = T.argmax(p_y_given_x, axis=0)
        return pred, p_y_given_x        
        
    def fit(self, X, y):
        init_theta, shapes = pack([self.W.get_value(), self.b.get_value()])
        opt_theta, opt_cost, d = sp.optimize.fmin_l_bfgs_b(
            func=self.cost_and_grad,
            x0=init_theta,
            fprime=None,
            args=(shapes, X, y),
            maxiter=self.max_iter,
            iprint=1)
        opt_param_list = unpack(opt_theta, shapes)
        for shared_param, opt_param in zip(self.params, opt_param_list):
             shared_param.set_value(opt_param)      

    def transform(self, X):
        return self.run_model(X)
    
    def evaluate(self, X, y):
        return self.test_model(X, y)
        
    def cost_and_grad(self, theta, *args):
        shapes = args[0]
        X = args[1]
        y = args[2]
        param_list = unpack(theta, shapes)
        cost =  self.cost_func(*(param_list + [X, y]))
        grad = self.grad_func(*(param_list + [X, y]))
        grad, _ = pack(grad)
        return cost, grad        
    
    def softmax(self, M):
        """
        normalise along the vertical axis
        """
        e_M = T.exp(M - M.max(axis=0, keepdims=True))
        return e_M / e_M.sum(axis=0, keepdims=True)

#### DNN class
(TODO: fine-tuning equations)

In [ ]:
#coding=utf-8
import theano
import theano.tensor as T
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import gzip
import cPickle

#from param_util import pack, unpack
#from autoencoder import AutoEncoder
#from softmax_regression import SoftmaxRegression

# for debugging
theano.config.optimizer='fast_run'#'fast_compile'
theano.config.exception_verbosity="high"

class DNN(object):
    """
    stacked autoencoder with a softmax output layer
    """
    def __init__(self, ae_layers, softmax_layer, finetune_max_iter=100, prefix='dnn'):
        self.ae_layers = ae_layers
        self.softmax_layer = softmax_layer
        self.finetune_max_iter = finetune_max_iter
        
        X = T.dmatrix(name=prefix+'_X')
        y = T.lvector(name=prefix+'y')
        
        # forward propagation through AE layers
        self.params = []        
        activations = [X]    
        for layer in self.ae_layers:
            activations += [layer.forward(activations[-1])]
            self.params += layer.hidden_layer.params
        
        # forward progagation through output layer
        pred, p_y_given_x = self.softmax_layer.predict(activations[-1])
        self.params += softmax_layer.params
        
        # cost
        cost = self.softmax_layer.calc_cost(p_y_given_x, y)
        # error
        error = 1.0*T.sum(T.neq(pred, y))/X.shape[1]
        
        grads = T.grad(cost, self.params)
        params = [p.type() for p in self.params]
        
        self.cost_func = theano.function(
            inputs=params + [X, y],
            outputs=cost,
            givens=[(s, p) for s, p in zip(self.params, params)])
        self.grad_func = theano.function(
            inputs=params + [X, y],
            outputs=grads,
            givens=[(s, p) for s, p in zip(self.params, params)])
        self.test_model = theano.function(
            inputs=[X, y],
            outputs=error)
        self.run_model = theano.function(
            inputs=[X],
            outputs=pred)
            
    def pretrain(self, X, y):
        """
        X: np.ndarray, each column stores a sample
        """
        X_in = X
        for layer in self.ae_layers:
            layer.fit(X_in)
            X_out = layer.transform(X_in)
            X_in = X_out
            
        self.softmax_layer.fit(X_in, y)
    
    def finetune(self, X, y):
        init_theta, shapes = pack(map(lambda p: p.get_value(), self.params))
        opt_theta, opt_cost, d = sp.optimize.fmin_l_bfgs_b(
            func=self.cost_and_grad,
            x0=init_theta,
            fprime=None,
            args=(shapes, X, y),
            maxiter=self.finetune_max_iter,
            iprint=1)
        opt_param_list = unpack(opt_theta, shapes)
        for shared_param, opt_param in zip(self.params, opt_param_list):
             shared_param.set_value(opt_param)
                 
    def fit(self, X, y):
        self.pretrain(X, y)
        self.finetune(X, y)
    
    def transform(self, X):
        return self.run_model(X)
  
    def evaluate(self, X, y):
        return self.test_model(X, y)

    def cost_and_grad(self, theta, *args):
        """
        compute the cost and gradient at theta in the parameter space
        args should be (shapes, X, y)
        X: each column stores a sample
        """
        shapes = args[0]
        X = args[1]
        y = args[2]
        param_list = unpack(theta, shapes)
        cost =  self.cost_func(*(param_list+[X, y]))
        grad = self.grad_func(*(param_list+[X, y]))
        grad, _ = pack(grad)
        return cost, grad

#### Evaluating the deep network for digit classification

In [ ]:
if __name__ == '__main__':
    np.random.seed(0)
    
    f = gzip.open('data/mnist.pkl.gz')
    train_set, valid_set, test_set = cPickle.load(f)
    
    # train model
    train_X = train_set[0].transpose()
    train_y = train_set[1]
    valid_X = valid_set[0].transpose()
    valid_y = valid_set[1]
    
    train_X = np.hstack((train_X, valid_X))
    train_y = np.hstack((train_y, valid_y))
    
    train_X = train_X[:, 0:60001:5]
    train_y = train_y[0:60001:5]
    
    n_in = train_X.shape[0]
    n_out = np.unique(train_y).shape[0]
    print "{n} training samples of dim {d}".format(n=train_X.shape[1], d=n_in)
    
    auto_encoder1 = AutoEncoder(
        n_in=train_X.shape[0],
        n_hid=200,
        L2_reg_coef=3e-3,
        sparse_reg_coef=3.0,
        sparse_rho=0.1,
        max_iter=200)  
    auto_encoder2 = AutoEncoder(
        n_in=auto_encoder1.n_hid,
        n_hid = 200,
        L2_reg_coef=3e-3,
        sparse_reg_coef=3.0,
        sparse_rho=0.1,
        max_iter=200) 
    softmax_regression = SoftmaxRegression(
        n_in=auto_encoder2.n_hid, 
        n_out=10, 
        L2_reg_coef=3e-3, 
        max_iter=100)
    dnn = DNN((auto_encoder1, auto_encoder2), softmax_regression, finetune_max_iter=100)
    dnn.fit(train_X, train_y)
    
    # test model
    test_X = test_set[0].transpose()
    test_y = test_set[1]
    #pred = dnn.transform(test_X)
    #error = 1.0*(pred != test_y).sum()/test_y.shape[0]
    error = dnn.evaluate(test_X, test_y)
    print 'error rate on test set is {e}%, accuracy is {a}%'.format(e=error*100, a=100-100*error)

#### Experimental results

Training set:
* 60,000 handwritten digit images of 0~9,
* Selecting a subset of the training samples by picking one for every five samples.

Test set:
* 10,000 handwritten digit images of 0~9.

<table>
<tbody>
<tr><td>Method</td><td>Accuracy</td></tr>
<tr><td>Baseline</td><td>9.8%</td></tr>
<tr><td>Softmax Regression</td><td>91.5%</td></tr>
<tr><td>DNN (1 AE)(pretraining)</td><td>92.4%</td></tr>
<tr><td>DNN (1 AE)(fine-tuning)</td><td>93.83%</td></tr>
<tr><td>DNN (1 AE)(pretaining plus fine-tuning)</td><td>95.0%</td></tr>
<tr><td>DNN (2 AEs)(pretraining)</td><td>85.9%</td></tr>
<tr><td>DNN (2 AEs)(fine-tuning)</td><td>93.77%</td></tr>
<tr><td>DNN (2 AEs)(pretaining plus fine-tuning)</td><td>95.9%</td></tr>
</table>
</tbody>


### Insights into the Behaviours of the Neurons

Find input $x$ that maximally activates a specific hidden neuron and has limited $L_2$ norm: 
$$
\max_{x}a^{(l)}_i(x)\\
s.t. \Vert x \Vert^2 \le 1
$$
This problem can be approximated by a non-constrained minimisation problem:
$$
\min_{x}\left[-a^{(l)}_i(x) + \lambda \max(\Vert x \Vert^2 - 1, 0)\right]
$$
where $\lambda$ is a fairly large number.

$\mathcal{l}(x)=\max(\Vert x \Vert^2 - 1, 0)$ can be considered as a hinge loss. Since the derivative of the hinge loss at $\Vert x \Vert^2 = 1$ is non-deterministic, smoothed versions may be preferred for more stable optimisation.

#### Hinge loss function

$\mathcal{l}(x)=\max(\Vert x \Vert^2 - 1, 0)$

As the optimal solution may very well lie in the neighborhood of $\Vert x \Vert^2 = 1$, the loss gradient therein should become more "cautious" as it approaches the border to avoid bouncing back and forth in vain.

#### Smoothed hinge loss function

$\mathcal{l}_s(x)=\frac{1}{2\gamma}\max(\Vert x \Vert^2 - 1, 0)^2$

![image](hinge_loss_plot.png)

In [1]:
# visulisation the first hidden layer
plt.close('all')
def rescale(x):
    # Remove DC (mean of images)
    x = x - x.mean()
    # Truncate to +/-3 standard deviations and scale to -1 to 1
    pstd = 3 * x.std()
    x = np.maximum(np.minimum(x, pstd), -pstd) / pstd
    # Rescale from [-1,1] to [0.1,0.9]
    x = (x + 1) * 0.4 + 0.1;
    return x
fig = plt.figure(0)
k = 0
for i in range(0, 64):#for i in np.random.randint(0, dnn.ae_layers[0].n_hid, 64):
    x = T.dvector('x')
    x0 = np.random.rand(28*28)
    x0 = x0 / np.sqrt((x0**2).sum())
    a = dnn.ae_layers[0].forward(x)
    # cost
    cost = -a[i] + 1e2 * T.max([0, (x**2).sum()-1])**2
    grad = T.grad(cost, x)
    cost_func = theano.function([x], cost)
    grad = T.grad(cost, x)
    grad_func = theano.function([x], grad)
    act_func = theano.function([x], a)
    def cost_and_grad(x):
        return cost_func(x), grad_func(x)
    opt_x, opt_cost, d = sp.optimize.fmin_l_bfgs_b(
        func=cost_and_grad,
        x0=x0,
        maxiter=1000,
        iprint=0)
    print ("||x||^2 is {v:.2f}, activation is {a:.2f}"
            .format(v=(opt_x**2).sum(), a=act_func(opt_x)[i]))
    opt_x_rescaled = rescale(opt_x)
    img = opt_x_rescaled.reshape((28,28))
    k = k + 1
    ax = fig.add_subplot(8, 8, k)
    ax.imshow(img, cmap='gray', interpolation='none')
    ax.set_axis_off()
plt.show()
fig.savefig('visual_dnn_layer1.png', format='png')

# visulisation the second hidden layer
fig = plt.figure(1)
k = 0
for i in range(0, 64):#np.random.randint(0, dnn.ae_layers[1].n_hid, 64):
    x = T.dvector('x')
    x0 = np.random.rand(28*28)
    x0 = x0 / np.sqrt((x0**2).sum())
    a1 = dnn.ae_layers[0].forward(x)
    a = dnn.ae_layers[1].forward(a1)
    # cost
    cost = -a[i] + 1e2 * T.max([0, (x**2).sum()-1])**2
    grad = T.grad(cost, x)
    cost_func = theano.function([x], cost)
    grad = T.grad(cost, x)
    grad_func = theano.function([x], grad)
    act_func = theano.function([x], a)
    def cost_and_grad(x):
        return cost_func(x), grad_func(x)
    opt_x, opt_cost, d = sp.optimize.fmin_l_bfgs_b(
        func=cost_and_grad,
        x0=x0,
        maxiter=1000,
        iprint=0)
    #print opt_x
    print ("||x||^2 is {v:.2f}, activation is {a:.2f}"
            .format(v=(opt_x**2).sum(), a=act_func(opt_x)[i]))        
    opt_x_rescaled = rescale(opt_x)
    img = opt_x_rescaled.reshape((28,28))
    k = k + 1
    ax = fig.add_subplot(8, 8, k)
    ax.imshow(img, cmap='gray', interpolation='none')
    ax.set_axis_off()
plt.show()
fig.savefig('visual_dnn_layer2.png', format='png')  

NameError: name 'T' is not defined

Inputs that maximise the activation of the first 64 neursons in the first hidden layer after pretraining:
![image](visual_dnn_layer1_pretrain.png)

Inputs that maximise the activation of of the first 64 neursons in the second hidden layer after pretraining:
![image](visual_dnn_layer2_pretrain.png)

Inputs that maximise the activation of the first 64 neursons in the first hidden layer after fine-tuning:
![image](visual_dnn_layer1_finetune.png)

Inputs that maximise the activation of of the first 64 neursons in the second hidden layer after fine-tuning:
![image](visual_dnn_layer2_finetune.png)

#### Generative Model v.s. Discriminative Model
* Generative Model
    * Tries to model the distribution of patterns in the feature space
    * More explainable
* Discriminative Model
    * Tries to model the differences among patterns
    * Less explainable

An example is to recognise two handwritten digits $1$ and $2$.